In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns 
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

np.random.seed(47)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
physical_devices = tf.config.experimental.list_physical_devices( 'GPU' )
print( 'Num GPUs Available: ', len( physical_devices ) )
if len( physical_devices ) > 0:
    tf.config.experimental.set_memory_growth( physical_devices[0], True )

Num GPUs Available:  0


In [3]:
experiment_files = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A52', 'A56', 'A61', 'A62', 'A63', 'A64', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A74', 'A81', 'A85', 'A89', 'A91', 'A99', 'A109', 'A110', 'A112', 'A113', 'A114', 'A115', 'A127', 'A129', 'A136', 'A137', 'A138', 'A139', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A152', 'A153', 'A154', 'A155', 'A156', 'A157', 'A159']

In [4]:
posetnet_dataset_path = "https://raw.githubusercontent.com/digitacs/4dv652-ml/main/datasets/marked_start_mid_end/"

## Reading Training Datasets

In [5]:
dataset = None

for file in experiment_files:
    posenet_data = pd.read_csv(posetnet_dataset_path+'{}.csv'.format( file ))
    
    if dataset is None:
        dataset = posenet_data
    else:
        dataset = pd.concat((dataset, posenet_data),ignore_index=True)

In [6]:
dataset['status'][dataset['status']==0] = 'Start'
dataset['status'][dataset['status']==1] = 'None'
dataset['status'][dataset['status']==2] = 'End'
dataset.drop(columns=['FrameNo'], inplace=True)
dataset['status'].fillna('None', inplace=True)

<ipython-input-6-663a43176429>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['status'][dataset['status']==0] = 'Start'


### Examine the class label imbalance

In [7]:
train_gc = dataset.groupby(['status']).size()
print(
    'Examples:\n    Total: {}\n    Start: {} ({:.2f}% of total)\n    End: {} ({:.2f}% of total)\n    None: {} ({:.2f}% of total)\n'
    .format(
      len(dataset), 
      train_gc['Start'],
      train_gc['Start'] / len(dataset),
      train_gc['End'],
      train_gc['End'] / len(dataset),
      train_gc['None'],
      train_gc['None'] / len(dataset)
    )
  )

Examples:
    Total: 19417
    Start: 4667 (0.24% of total)
    End: 4311 (0.22% of total)
    None: 10439 (0.54% of total)



### Encoding and Scaling

In [8]:
target_data = pd.get_dummies(dataset['status'], prefix='is')
input_data = dataset.drop(columns=['status'])

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(input_data)
input_data = scaler.transform(input_data)

## Reading Test Datasets

In [10]:
test_dataset = None

for file_no in range(1,23):
    posenet_data = pd.read_csv(posetnet_dataset_path+'B{}.csv'.format( file_no ))
    
    if dataset is None:
        test_dataset = posenet_data
    else:
        test_dataset = pd.concat((test_dataset, posenet_data),ignore_index=True)

In [11]:
test_dataset['status'][test_dataset['status']==0] = 'Start'
test_dataset['status'][test_dataset['status']==1] = 'None'
test_dataset['status'][test_dataset['status']==2] = 'End'
test_dataset.drop(columns=['FrameNo'], inplace=True)
test_dataset['status'].fillna('None', inplace=True)

<ipython-input-11-cbad75cf242a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['status'][test_dataset['status']==0] = 'Start'


### Examine the class label imbalance

In [12]:
train_gc = test_dataset.groupby(['status']).size()
print(
    'Examples:\n    Total: {}\n    Start: {} ({:.2f}% of total)\n    End: {} ({:.2f}% of total)\n    None: {} ({:.2f}% of total)\n'
    .format(
      len(test_dataset), 
      train_gc['Start'],
      train_gc['Start'] / len(test_dataset),
      train_gc['End'],
      train_gc['End'] / len(test_dataset),
      train_gc['None'],
      train_gc['None'] / len(test_dataset)
    )
  )

Examples:
    Total: 5395
    Start: 2102 (0.39% of total)
    End: 1405 (0.26% of total)
    None: 1888 (0.35% of total)



### Encoding and Scaling

In [13]:
test_target_data = pd.get_dummies(test_dataset['status'], prefix='is')
test_input_data = test_dataset.drop(columns=['status'])

In [14]:
test_input_data = scaler.transform(test_input_data)

## Create a model

### The model **configuration**

In [15]:
from keras.metrics import TruePositives,FalsePositives,TrueNegatives,FalseNegatives,CategoricalAccuracy,Precision,Recall,AUC
from keras.callbacks import EarlyStopping

METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      CategoricalAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='acc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]

TEST_ID = 38
ACTIVATION = 'exponential'
EPOCHS = 150
BATCH_SIZE = 64
OPTIMIZER = 'rmsprop'
LOSS = 'categorical_crossentropy'


early_stopping = EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [16]:
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, BatchNormalization, MaxPooling2D, Flatten

def make_model(optimizer = OPTIMIZER ,loss = LOSS , metrics = METRICS, output_size=3):
  model = Sequential()
  model.add(Dense( units=26, input_dim=26, activation='relu' ))
  model.add(Dense(32, activation = ACTIVATION, kernel_initializer = 'he_uniform'))
  model.add(Dense(32, activation = ACTIVATION, kernel_initializer = 'he_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(output_size, activation = 'softmax'))


  # Todo: check other optimizer like 'adam' and 'nadam' as well
  model.compile(optimizer = optimizer ,loss = loss, metrics = metrics)
  print(model.summary())

  return model

In [17]:
base_model = make_model()

history = base_model.fit( 
    x=input_data, 
    y=target_data, 
    validation_split=0.1, 
    shuffle=True, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=2 )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 26)                702       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                864       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 2,721
Trainable params: 2,721
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150


InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential/dense_3/Softmax:0) = ] [[-nan -nan -nan]...] [y (Cast_3/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]] [Op:__inference_train_function_3369]

Function call stack:
train_function


In [ ]:
training_history = pd.DataFrame(history.history)
training_history['epochs'] = range(len(training_history['accuracy']))

In [ ]:
sns.set_theme(style="darkgrid")
sns.color_palette("husl", 8)

### Training History

In [ ]:
PLOTS = [
           {'cols':['tp', 'val_tp'], 'title':'Training and validation TruePositives', 'yLabel':'TruePositives'},
           {'cols':['fp', 'val_fp'], 'title':'Training and validation FalsePositives', 'yLabel':'FalsePositives'},
           {'cols':['tn', 'val_tn'], 'title':'Training and validation TrueNegatives', 'yLabel':'TrueNegatives'},
           {'cols':['fn', 'val_fn'], 'title':'Training and validation FalseNegatives', 'yLabel':'FalseNegatives'},
           {'cols':['accuracy', 'val_accuracy'], 'title':'Training and validation CategoricalAccuracy', 'yLabel':'CategoricalAccuracy'},
           {'cols':['precision', 'val_precision'], 'title':'Training and validation Precision', 'yLabel':'Precision'},
           {'cols':['recall', 'val_recall'], 'title':'Training and validation Recall', 'yLabel':'Recall'},
           {'cols':['acc', 'val_acc'], 'title':'Training and validation AUC', 'yLabel':'AUC'},
           {'cols':['prc', 'val_prc'], 'title':'Training and validation PRC', 'yLabel':'PRC'} # precision-recall curve
]

# Testing the Model

### Overal Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = base_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( test_target_data, predictions ),
}

acc_a = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_a)

### Start and End Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = base_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])
bool_test_not_none = np.array(test_target_data['is_None']) != 1
y_prod = pd.DataFrame(predictions[ bool_test_not_none])
y_true = pd.DataFrame(test_target_data[ bool_test_not_none])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( y_true, y_prod ),
}

acc_b = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_b)

In [ ]:
fig = plt.figure(figsize=(20,20))
fig_no = 331
model_type = 'Base Model'
title = 'Model #{} ({})\n Overal {}  -  Strat.Stop {}\n Activation function: {}\n Numner of epochs: {} / Batch size: {}\nOptimizer: {} / Loss: {}'.format(TEST_ID, model_type,acc_a,acc_b, ACTIVATION, EPOCHS,BATCH_SIZE,OPTIMIZER,LOSS)
fig.suptitle(title, fontsize="x-large")

for i in range(len(PLOTS)):
    ax = fig.add_subplot(str(fig_no))
    sns.lineplot(data=training_history[PLOTS[i]['cols']], ax=ax)
    plt.title(PLOTS[i]['title'])
    plt.xlabel('Epochs')
    plt.ylabel(PLOTS[i]['yLabel'])
    fig_no += 1
    
plt.savefig('Model-{}({})'.format(TEST_ID, model_type))
plt.show()

# Solution-1: Class weighting
These will cause the model to "pay more attention" to examples from an under-represented class

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
train_gc = dataset.groupby(['status']).size()
total = len(dataset)

weight_for_Start = (1 / train_gc['Start'])*(total)/3.0 
weight_for_End = (1 / train_gc['End'])*(total)/3.0
weight_for_None = (1 / train_gc['None'])*(total)/3.0

class_weight = {0: weight_for_Start, 1: weight_for_End, 2:weight_for_None}

print('Weight for class Start: {:.2f}'.format(weight_for_Start))
print('Weight for class End: {:.2f}'.format(weight_for_End))
print('Weight for class None: {:.2f}'.format(weight_for_None))

In [ ]:
weighted_model = make_model()

weighted_history = weighted_model.fit(
    x=input_data, 
    y=target_data, 
    validation_split=0.1, 
    shuffle=True, 
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=2,
    # The class weights go here
    class_weight=class_weight) 

### Overal Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = weighted_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( test_target_data, predictions ),
}

acc_a = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_a)

### Start and End Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = weighted_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])
bool_test_not_none = np.array(test_target_data['is_None']) != 1
y_prod = pd.DataFrame(predictions[ bool_test_not_none])
y_true = pd.DataFrame(test_target_data[ bool_test_not_none])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( y_true, y_prod ),
}

acc_b = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_b)

In [ ]:
weighted_history = pd.DataFrame(weighted_history.history)
weighted_history['epochs'] = range(len(weighted_history['accuracy']))

fig = plt.figure(figsize=(20,20))
fig_no = 331
model_type = 'Class Weighting'
title = 'Model #{} ({})\n Overal {}  -  Strat.Stop {}\n Activation function: {}\n Numner of epochs: {} / Batch size: {}\nOptimizer: {} / Loss: {}'.format(TEST_ID, model_type,acc_a,acc_b, ACTIVATION, EPOCHS,BATCH_SIZE,OPTIMIZER,LOSS)
fig.suptitle(title, fontsize="x-large")

for i in range(len(PLOTS)):
    ax = fig.add_subplot(str(fig_no))
    sns.lineplot(data=training_history[PLOTS[i]['cols']], ax=ax)
    plt.title(PLOTS[i]['title'])
    plt.xlabel('Epochs')
    plt.ylabel(PLOTS[i]['yLabel'])
    fig_no += 1
    
plt.savefig('Model-{}({})'.format(TEST_ID, model_type))
plt.show()

#  Solution-2: Oversampling

In [ ]:
bool_train_not_none = np.array(target_data['is_None']) != 1

pos_features = input_data[bool_train_not_none]
neg_features = input_data[~bool_train_not_none]

pos_labels = target_data[bool_train_not_none]
neg_labels = target_data[~bool_train_not_none]

Fill choices array randomly utill it become the same size as negative labels

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels.iloc[choices]

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

In [ ]:
resampled_model = make_model()

resampled_history = resampled_model.fit(
    x=resampled_features,
    y=resampled_labels,
    validation_split=0.1, 
    shuffle=True, 
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=2)

### Overal Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = resampled_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( test_target_data, predictions ),
}

acc_a = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_a)

### Start and End Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = resampled_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])
bool_test_not_none = np.array(test_target_data['is_None']) != 1
y_prod = pd.DataFrame(predictions[ bool_test_not_none])
y_true = pd.DataFrame(test_target_data[ bool_test_not_none])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( y_true, y_prod ),
}

acc_b = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_b)

In [ ]:
resampled_history = pd.DataFrame(resampled_history.history)
resampled_history['epochs'] = range(len(resampled_history['accuracy']))

fig = plt.figure(figsize=(20,20))
fig_no = 331
model_type = 'Over Sampling'
title = 'Model #{} ({})\n Overal {}  -  Strat.Stop {}\n Activation function: {}\n Numner of epochs: {} / Batch size: {}\nOptimizer: {} / Loss: {}'.format(TEST_ID, model_type,acc_a,acc_b, ACTIVATION, EPOCHS,BATCH_SIZE,OPTIMIZER,LOSS)
fig.suptitle(title, fontsize="x-large")

for i in range(len(PLOTS)):
    ax = fig.add_subplot(str(fig_no))
    sns.lineplot(data=training_history[PLOTS[i]['cols']], ax=ax)
    plt.title(PLOTS[i]['title'])
    plt.xlabel('Epochs')
    plt.ylabel(PLOTS[i]['yLabel'])
    fig_no += 1
    
plt.savefig('Model-{}({})'.format(TEST_ID, model_type))
plt.show()

#  Solution-3: Focal loss

In [ ]:
target_data_focal = target_data.astype(float)

In [ ]:
import keras.backend as K
import tensorflow as tf

def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha*((1-p)^gamma)*log(p)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediction value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -y_true*K.log(y_pred)
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = alpha * y_true * K.pow((1-y_pred), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss

focal_loss_model = make_model(loss=categorical_focal_loss(gamma=3))

focal_loss_history = focal_loss_model.fit( 
    x=input_data, 
    y=target_data_focal, 
    validation_split=0.1, 
    shuffle=True, 
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=2)

### Overal Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = focal_loss_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])
results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( test_target_data, predictions ),
}

acc_a = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_a)

### Start and End Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, f1_score

predictions = focal_loss_model.predict( x=test_input_data, batch_size=BATCH_SIZE, verbose=1 )
predictions = pd.DataFrame(predictions.round().astype(int), columns=['is_End',	'is_None',	'is_Start'])
bool_test_not_none = np.array(test_target_data['is_None']) != 1
y_prod = pd.DataFrame(predictions[ bool_test_not_none])
y_true = pd.DataFrame(test_target_data[ bool_test_not_none])

results = { 
    'name': 'Sigmoid',
    'accuracy score': accuracy_score( y_true, y_prod ),
}

acc_b = 'Accuracy: {:.2f}%'.format( results['accuracy score']*100 ) 
print(acc_b)

In [ ]:
focal_loss_history = pd.DataFrame(focal_loss_history.history)
focal_loss_history['epochs'] = range(len(focal_loss_history['accuracy']))

fig = plt.figure(figsize=(20,20))
fig_no = 331
model_type = 'Focal Loss'
title = 'Model #{} ({})\n Overal {}  -  Strat.Stop {}\n Activation function: {}\n Numner of epochs: {} / Batch size: {}\nOptimizer: {} / Loss: {}'.format(TEST_ID, model_type,acc_a,acc_b, ACTIVATION, EPOCHS,BATCH_SIZE,OPTIMIZER,LOSS)
fig.suptitle(title, fontsize="x-large")

for i in range(len(PLOTS)):
    ax = fig.add_subplot(str(fig_no))
    sns.lineplot(data=focal_loss_history[PLOTS[i]['cols']], ax=ax)
    plt.title(PLOTS[i]['title'])
    plt.xlabel('Epochs')
    plt.ylabel(PLOTS[i]['yLabel'])
    fig_no += 1
    
plt.savefig('Model-{}({})'.format(TEST_ID, model_type))
plt.show()